In [1]:
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    "FASTX"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

[ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
[ Info: Skipping precompilation since __precompile__(false). Importing Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd].


In [2]:
project_dir = dirname(pwd())
data_dir = mkpath(joinpath(project_dir, "data"))

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data"

In [3]:
viral_reference_fastas_dir = mkpath(joinpath(data_dir, "viral-reference-fastas"))

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas"

In [4]:
blastdb_dir = joinpath(homedir(), "workspace", "blastdb")

"/home/cjprybol/workspace/blastdb"

In [5]:
reference_fastas = sort(filter(x -> occursin(r"\.f(na|asta)\.gz$", x), readdir(viral_reference_fastas_dir, join=true)), by=x->filesize(x))

6-element Vector{String}:
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/ictv-examplar-viruses.fna.gz"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/ref_viruses_rep_genomes.fasta.gz"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/GCF_000001405.40_GRCh38.p14_genomic.fna.gz"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/IMGVR_all_nucleotides-high_confidence.fna.gz"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/IMGVR_all_nucleotides-high_confidence.revised.fna.gz"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/nt_viruses.fasta.gz"

In [ ]:
# input_records = Mycelia.open_fastx("$(viral_reference_fastas_dir)/IMGVR_all_nucleotides-high_confidence.fna.gz")
# revised_record_iterator = (FASTX.FASTA.Record(first(split(FASTX.description(record), '|')), FASTX.sequence(record)) for record in input_records)
# IMGVR_shorted_descriptions = "$(viral_reference_fastas_dir)/IMGVR_all_nucleotides-high_confidence.revised.fna.gz"
# # 25728.690161 seconds
# @time Mycelia.write_fasta(outfile = IMGVR_shorted_descriptions, gzipped = true, records = revised_record_iterator)

In [6]:
reference_fasta = "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/IMGVR_all_nucleotides-high_confidence.revised.fna.gz"

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/IMGVR_all_nucleotides-high_confidence.revised.fna.gz"

In [8]:
# for reference_fasta in reference_fastas
database_title = replace(basename(reference_fasta), r"\.f(na|asta)\.gz" => "")
blastdb = joinpath(blastdb_dir, database_title)
@show blastdb
hits = filter(x -> occursin(blastdb, x), readdir(blastdb_dir, join=true))
# @show hits
if isempty(hits)
    @show "make me"
    @time run(pipeline(`gzip -dc $(reference_fasta)`, `$(Mycelia.MAMBA) run --live-stream -n blast makeblastdb -title $(database_title) -in - -dbtype nucl -parse_seqids -out $(blastdb)`))
else
    @show "done"
end
# end

blastdb = "/home/cjprybol/workspace/blastdb/IMGVR_all_nucleotides-high_confidence.revised"
"make me" = "make me"


Building a new DB, current time: 01/30/2024 15:56:45
New DB name:   /home/cjprybol/workspace/blastdb/IMGVR_all_nucleotides-high_confidence.revised
New DB title:  IMGVR_all_nucleotides-high_confidence.revised
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B




BLAST Database creation error: Error: Duplicate seq_ids are found: 
LCL|IMGVR_UVIG_GVMAG-M-2140918012-1_000001

ERROR conda.cli.main_run:execute(124): `conda run makeblastdb -title IMGVR_all_nucleotides-high_confidence.revised -in - -dbtype nucl -parse_seqids -out /home/cjprybol/workspace/blastdb/IMGVR_all_nucleotides-high_confidence.revised` failed. (See above for error)


LoadError: failed process: Process(`[4m/home/cjprybol/.julia/conda/3/x86_64/bin/mamba[24m [4mrun[24m [4m--live-stream[24m [4m-n[24m [4mblast[24m [4mmakeblastdb[24m [4m-title[24m [4mIMGVR_all_nucleotides-high_confidence.revised[24m [4m-in[24m [4m-[24m [4m-dbtype[24m [4mnucl[24m [4m-parse_seqids[24m [4m-out[24m [4m/home/cjprybol/workspace/blastdb/IMGVR_all_nucleotides-high_confidence.revised[24m`, ProcessExited(1)) [1]
